# Problem 59

### XOR decryption

Each character on a computer is assigned a unique code and the preferred standard is ASCII (American Standard Code for Information Interchange). For example, uppercase A = 65, asterisk (*) = 42, and lowercase k = 107.

A modern encryption method is to take a text file, convert the bytes to ASCII, then XOR each byte with a given value, taken from a secret key. The advantage with the XOR function is that using the same encryption key on the cipher text, restores the plain text; for example, 65 XOR 42 = 107, then 107 XOR 42 = 65.

For unbreakable encryption, the key is the same length as the plain text message, and the key is made up of random bytes. The user would keep the encrypted message and the encryption key in different locations, and without both "halves", it is impossible to decrypt the message.

Unfortunately, this method is impractical for most users, so the modified method is to use a password as a key. If the password is shorter than the message, which is likely, the key is repeated cyclically throughout the message. The balance for this method is using a sufficiently long password key for security, but short enough to be memorable.

Your task has been made easy, as the encryption key consists of three lower case characters. Using cipher.txt (right click and 'Save Link/Target As...'), a file containing the encrypted ASCII codes, and the knowledge that the plain text must contain common English words, decrypt the message and find the sum of the ASCII values in the original text.

### Solution

Since each of the three characters is a lowercase character, we have in total 26x26x26 = 17576 possible passwords.

Instead of trying all the combination, we can try finding, for each character, the character that maximizes the number of original english letters (lowercas and uppercase) and spaces.

In [1]:
import numpy as np

with open('data/p059_cipher.txt') as c:
    cipher = np.array([int(number) for number in c.read().split(',')])

In [2]:
def decipher(cipher, password):
    
    idx = 0
    pwd = len(password)
    
    for ascii_char in cipher:
        yield ascii_char ^ password[idx]
        idx = (idx + 1) % pwd

In [3]:
def decipher_as_text(cipher, password):
    
    text = [chr(c) for c in decipher(cipher, password)]
    return ''.join(text)

We define a dictionary of weights 

In [4]:
weights = {}
for n in xrange(ord('a'), ord('z') + 1):
    weights[n] = 1.

for n in xrange(ord('A'), ord('Z') + 1):
    weights[n] = 1.
    
weights[ord(' ')] = 1.

For each password digit, we take the ciphered characters affacted by the character and we try to maximize the score computed by comparing the deciphered text with the weights dictionary.

In [5]:
for idx in range(3):
    
    max_score = 0
    best_pwd_char = -1
    
    for pwd_char in xrange(ord('a'), ord('z') + 1):
    
        affected_ascii_chars = np.array(cipher[idx::3])
        deciphered_partial = affected_ascii_chars ^ pwd_char        
        curr_score = sum(weights.get(n, 0.) for n in deciphered_partial)
        
        if curr_score > max_score:
            max_score = curr_score
            best_pwd_char = pwd_char
            
    print idx, best_pwd_char, max_score

0 103 376.0
1 111 391.0
2 100 380.0


I think we guessed it :)

In [6]:
decipher_as_text(cipher, [103, 111, 100])  # god

"(The Gospel of John, chapter 1) 1 In the beginning the Word already existed. He was with God, and he was God. 2 He was in the beginning with God. 3 He created everything there is. Nothing exists that he didn't make. 4 Life itself was in him, and this life gives light to everyone. 5 The light shines through the darkness, and the darkness can never extinguish it. 6 God sent John the Baptist 7 to tell everyone about the light so that everyone might believe because of his testimony. 8 John himself was not the light; he was only a witness to the light. 9 The one who is the true light, who gives light to everyone, was going to come into the world. 10 But although the world was made through him, the world didn't recognize him when he came. 11 Even in his own land and among his own people, he was not accepted. 12 But to all who believed him and accepted him, he gave the right to become children of God. 13 They are reborn! This is not a physical birth resulting from human passion or plan, this

In [7]:
sum(c for c in decipher(cipher, [103, 111, 100]))

107359